# Hoja de Trabajo 1

## Ejercicio 4 

### Inciso C

Utilizando cualquier software, resuelva el problema de optimización planteado en el inciso anterior. ¿Qué tipo de solución (global o local) encontró? Justifique su respuesta

In [1]:
# Para activar los gráficos interactivos
%matplotlib widget

# Paquetes a utilizar
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Habilitar la compatibilidad con tensorflow v1 si se tienen tensorflow v2
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Instructions for updating:
non-resource variables are not supported in the long term
Enabled compatitility to tf1.x


In [2]:
# Definición de función encargada de crear un grafo para hacer una regresión lineal
def Create_LinRegGraph(NumFilas, NumParams):

    # Se reinicia la creación del grafo creado
    tf.reset_default_graph()

    # Se crea un objeto de tipo grafo
    grafo = tf.Graph()

    # Se incluyen nodos dentro del grafo
    with grafo.as_default():

        # Input: Se definen los datos de entrenamiento
        # X: Tantas filas como datos (None para un tamaño variable). Tantas columnas como features
        # Y: Tantas filas como datos (None para un tamaño variable). 1 Columna
        X = tf.placeholder(tf.float32, [None, NumParams], "X")
        Y = tf.placeholder(tf.float32, [None, 1], "Y")

        # Input: Learning rate 
        learning_rate = tf.placeholder(dtype="float", name="lr")

        # Se inicializan los parámetros correspondientes a las pendientes (m's) y el bias
        params = tf.Variable(tf.zeros((NumParams, 1)), name="Theta", dtype=tf.float32)

        # Predicción de la salida dados los parámetros M y B
        with tf.name_scope("Predict"):
            Y_hat = tf.matmul(X, params)

        # Cálculo del error 
        with tf.name_scope("Error"):
            error = tf.reduce_sum(tf.pow(Y_hat - Y, 2)) / (2 * NumFilas)

            # Incluir el error en la parte de "Scalars" de tensorboard
            error_summary = tf.summary.scalar("Error", error)

        # Obtener el valor de los gradientes para M y B
        grads_params = tf.gradients(error, params)

        # La operación de gradiente es acoplada a un print para facilitar el debugging
        # grads_print = tf.tuple(grads_params, control_inputs=[tf.print(grads_params)])
        grads_print = tf.tuple(grads_params, control_inputs=[])

        # El gradiente retorna un tensor de más dimensiones de las que regresó. 
        # Ejemplo: Para un tensor de "params" 2D, tf.gradients retorna un tensor 3D con 2 elementos.
        # Se "aplanan" los datos para regresar a la forma original de params
        grads = tf.reshape(grads_print[0], tf.shape(params))

        # Actualizar los parámetros del algoritmo
        with tf.name_scope("Update"):
            delta_params = tf.assign(params, params - learning_rate * grads)

        # Inicializar variables globales
        init = tf.global_variables_initializer()

    return grafo, init, [X, Y], [learning_rate, delta_params, error_summary, params, error]

In [3]:
# Función de entrenamiento
def Train(x, y, lr, epochs, metadata):

    from sklearn.preprocessing import PolynomialFeatures

    # Se agrega una columna de 1's a X
    x = np.hstack((x, np.ones((x.shape[0],1))))

    # Se extraen las dimensiones de los datos de entrada
    num_fil = x.shape[0]
    num_col = x.shape[1]

    # Se crea un grafo nuevo con el número de columnas de X
    grafo, init, linRegIO, Nodes = Create_LinRegGraph(num_fil, num_col)

    # Datos de entrenamiento (X) y labels (Y)
    X, Y = linRegIO

    # Se extrae la definición de los diferentes nodos utilizados luegos por TF
    learning_rate = Nodes[0]
    delta_params = Nodes[1]
    error_summary = Nodes[2]
    params = Nodes[3]
    costo = Nodes[4]

    with tf.Session(graph = grafo) as sess:
        
        # Inicializa todas las variables de ser necesario
        tf.initialize_all_variables().run()

        # Crea un directorio para tensorboard
        # Generalmente el directorio "padre" de tensorboard es "graphs" pero puede cambiarse
        writer = tf.summary.FileWriter((f'./graphs/{metadata}  model_lr={str(lr)}, epochs={str(epochs)}, no_feat={str(num_col)}'), sess.graph)

        # Inicializar el grafo
        sess.run(init)

        # Se definen los inputs del grafo
        inputs_grafo = {
            X: x,
            Y: y,
            learning_rate: lr
        }

        # Iterar para cada Epoch
        for epoch in range(epochs):
            
            # Se corre el grafo con los datos elegidos
            sess.run(delta_params, feed_dict=inputs_grafo)

            # Se agregan todos los escalares al tensorboard
            e = sess.run(error_summary, feed_dict=inputs_grafo)
            writer.add_summary(e, epoch)
        
        # Se extraen los parámetros resultantes de la regresión y el error
        theta = params.eval()
        error = costo.eval(feed_dict=inputs_grafo)
            
        # Finalizar el "writer" hacia tensor board
        writer.close()

    return theta, error

In [23]:
# Se convierten los arrays unidimensionales de numpy en arrays bidimensionales
x_train = np.array([[16, 140],
                    [25, 149],
                    [39, 165],
                    [45, 170],
                    [49, 165],
                    [64, 159],
                    [70, 144]])
y_train = np.array([[125, 129, 127, 150, 161, 144, 132]]).T

# Se realiza el entrenamiento
params, error = Train(x=x_train, y=y_train, lr=0.00001, epochs=500, metadata="HT1_4C")

In [24]:
# Se crea una figura 3D
fig = plt.figure()
ax = Axes3D(fig)

# Datos simulados de modelo lineal
x1_model = np.linspace(min(x_train[:,0]), max(x_train[:,0]) + 1, 100)
x2_model = np.linspace(min(x_train[:,1]), max(x_train[:,1]) + 1, 100)
y_model = params[0,0] * x1_model + params[1,0] * x2_model + params[2,0]

# Gráficas
data = ax.scatter(xs=x_train[:,0], ys=x_train[:,1], zs=y_train, label="Data")
model = ax.scatter(xs=x1_model, ys=x2_model, zs=y_model, c="r", s=0.8, label="Regresión")

# Títulos
plt.title("Resultado Regresión")
plt.xlabel("X1")
plt.ylabel("X2")
plt.legend(handles=[data, model])
plt.xticks(rotation=90)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …